<a href="https://colab.research.google.com/github/ttchengab/AdversarialDefense/blob/master/CleverhansTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y tensorflow-addons
!pip uninstall -y tensorflow-datasets
!pip uninstall -y tensorflow-estimator
!pip uninstall -y tensorflow-gcs-config
!pip uninstall -y tensorflow-hub
!pip uninstall -y tensorflow-privacy
!pip uninstall -y tensorflow-metadata
!pip uninstall -y tensorflow-probability
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.14
!pip install git+https://github.com/tensorflow/cleverhans.git#egg=cleverhans

Uninstalling tensorflow-addons-0.8.3:
  Successfully uninstalled tensorflow-addons-0.8.3
Uninstalling tensorflow-datasets-2.1.0:
  Successfully uninstalled tensorflow-datasets-2.1.0
Uninstalling tensorflow-estimator-2.3.0:
  Successfully uninstalled tensorflow-estimator-2.3.0
Uninstalling tensorflow-gcs-config-2.3.0:
  Successfully uninstalled tensorflow-gcs-config-2.3.0
Uninstalling tensorflow-hub-0.8.0:
  Successfully uninstalled tensorflow-hub-0.8.0
Uninstalling tensorflow-privacy-0.2.2:
  Successfully uninstalled tensorflow-privacy-0.2.2
Uninstalling tensorflow-metadata-0.22.2:
  Successfully uninstalled tensorflow-metadata-0.22.2
Uninstalling tensorflow-probability-0.11.0:
  Successfully uninstalled tensorflow-probability-0.11.0
Uninstalling tensorflow-2.3.0:
  Successfully uninstalled tensorflow-2.3.0
     |████████████████████████████████| 377.0MB 44kB/s 
     |████████████████████████████████| 491kB 45.5MB/s 
     |████████████████████████████████| 3.2MB 50.0MB/s 
     |███████

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import warnings
import numpy as np
import tensorflow as tf
from tensorflow import keras
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms

from cleverhans.attacks import FastGradientMethod
from cleverhans.compat import flags
from cleverhans.train import train
from cleverhans.dataset import MNIST
from cleverhans.model import CallableModelWrapper
from cleverhans.loss import CrossEntropy
from cleverhans.utils import AccuracyReport
from cleverhans.utils_tf import model_eval
from cleverhans.utils_pytorch import convert_pytorch_model_to_tf

#change to completely pytorch
from cleverhans.future.torch.attacks.fast_gradient_method import fast_gradient_method

#keras libraries, remove later
# from cleverhans.utils_keras import cnn_model
# from cleverhans.utils_keras import KerasModelWrapper

In [5]:
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

In [6]:
class ResBlock(nn.Module):
    
    def __init__(self, in_size:int, hidden_size:int, out_size:int):
        super().__init__()
        self.conv1 = nn.Conv2d(in_size, hidden_size, 3, padding=1)
        self.conv2 = nn.Conv2d(hidden_size, out_size, 3, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(hidden_size)
        self.batchnorm2 = nn.BatchNorm2d(out_size)
    
    def convblock(self, x):
        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = F.relu(self.batchnorm2(self.conv2(x)))
        return x
    
    def forward(self, x): return x + self.convblock(x) # skip connection

class ResNet1(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.res1 = ResBlock(1, 8, 16)
        self.res2 = ResBlock(16, 32, 16)
        self.fc1 = nn.Linear(16 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, 10)
        
    def forward(self, x):
        #1x28x28
        x = self.res1(x)
        #16x28x28
        x = self.res2(x) 
        #16x28x28
        x = F.max_pool2d(F.relu(x), 2)
        #16x14x14
        x = x.view(-1, 16*14*14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x,dim=-1)

class ResNet2(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.res1 = ResBlock(1, 8, 12)
        self.res2 = ResBlock(12, 16, 12)
        self.fc1 = nn.Linear(12 * 14 * 14, 1024)
        self.fc2 = nn.Linear(1024, 10)
        
    def forward(self, x):
        #1x28x28
        x = self.res1(x)
        #32x28x28
        x = self.res2(x) 
        #16x28x28
        x = F.max_pool2d(F.relu(x), 2)
        #16x14x14
        x = x.view(-1, 12*14*14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x,dim=-1)

class ResNet3(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.res1 = ResBlock(1, 8, 32)
        self.fc1 = nn.Linear(32 * 14 * 14, 1024)
        self.fc2 = nn.Linear(1024, 10)
        
    def forward(self, x):
        #1x28x28
        x = self.res1(x)
        #32x28x28
        x = F.max_pool2d(F.relu(x), 2)
        #32x14x14
        x = x.view(-1, 32*14*14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x,dim=-1)

class ResNet4(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.res1 = ResBlock(1, 8, 16)
        self.res2 = ResBlock(16, 32, 16)
        self.res3 = ResBlock(16, 8, 16)
        self.fc1 = nn.Linear(16 * 14 * 14, 1024)
        self.fc2 = nn.Linear(1024, 10)
        
    def forward(self, x):
        #1x28x28
        x = self.res1(x)
        #16x28x28
        x = self.res2(x) 
        #32x28x28
        x = self.res3(x)
        #16x28x28
        x = F.max_pool2d(F.relu(x), 2)
        #16x14x14
        x = x.view(-1, 16*14*14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x,dim=-1)


class LeNet5(torch.nn.Module):          
     
    def __init__(self):     
        super(LeNet5, self).__init__()
        # Convolution (In LeNet-5, 32x32 images are given as input. Hence padding of 2 is done below)
        self.conv1 = torch.nn.Conv2d(1, 6, 5, padding=2)
        # Max-pooling
        self.max_pool_1 = torch.nn.MaxPool2d(kernel_size=2)
        # Convolution
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        
        # Fully connected layer
        self.fc1 = nn.Linear(16*5*5, 120)   
        self.fc2 = nn.Linear(120, 84)       
        self.fc3 = nn.Linear(84, 10)    
        
    def forward(self, x):
        # convolve, then perform ReLU non-linearity
        x = F.relu(self.conv1(x))  
        # max-pooling with 2x2 grid 
        x = F.max_pool2d(x, 2) 
        # convolve, then perform ReLU non-linearity
        x = F.relu(self.conv2(x))
        # max-pooling with 2x2 grid
        x = F.max_pool2d(x, 2)
        # first flatten 'max_pool_2_out' to contain 16*5*5 columns
        # read through https://stackoverflow.com/a/42482819/7551231
        x = x.view(-1, 16*5*5)
        # FC-1, then perform ReLU non-linearity
        x = F.relu(self.fc1(x))
        # FC-2, then perform ReLU non-linearity
        x = F.relu(self.fc2(x))
        # FC-3
        x = self.fc3(x)
        
        return F.log_softmax(x,dim=-1)

class PytorchMnistModel(nn.Module):
  def __init__(self):
    super(PytorchMnistModel, self).__init__()
    # input is 28x28
    # padding=2 for same padding
    self.conv1 = nn.Conv2d(1, 32, 5, padding=2)
    # feature map size is 14*14 by pooling
    # padding=2 for same padding
    self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
    # feature map size is 7*7 by pooling
    self.fc1 = nn.Linear(64 * 7 * 7, 1024)
    self.fc2 = nn.Linear(1024, 10)

  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), 2)
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, 64 * 7 * 7)  # reshape Variable
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return F.log_softmax(x, dim=-1)
  
class FCNet(nn.Module):
  def __init__(self):
    super(FCNet, self).__init__()
    self.conv1 = nn.Conv2d(1, 28, 5)
    self.conv2 = nn.Conv2d(28, 42, 3, padding = 1)
    self.fc1 = nn.Linear(42*6*6, 512)
    self.fc2 = nn.Linear(512, 10)
  
  def forward(self, x):
    #1, 28, 28
    x = F.max_pool2d(F.relu(self.conv1(x)), 2)
    #28, 12, 12
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    #42, 6, 6
    x = x.view(-1, 42*6*6)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return F.log_softmax(x, dim=-1)


class VGGNet(nn.Module):
   def __init__(self):
        super(VGGNet, self).__init__()
        self.conv11 = nn.Conv2d(1, 64, 3)
        self.conv12 = nn.Conv2d(64, 64, 3)
        self.conv21 = nn.Conv2d(64, 128, 3)
        self.conv22 = nn.Conv2d(128, 128, 3)
        self.fc1 = nn.Linear(128 * 2 * 2, 1024)
        self.fc2 = nn.Linear(1024, 10)
   def forward(self, x):
       #1, 28, 28
       x = F.relu(self.conv11(x))
       #64, 26, 26
       x = F.relu(self.conv12(x))
       #64, 24, 24
       x = F.max_pool2d(x, (2,2))
       #64, 12, 12
       x = F.relu(self.conv21(x))
       #128, 10, 10
       x = F.relu(self.conv22(x))
       #128, 8, 8
       x = F.max_pool2d(x, (2,2))
       #128, 4, 4
       x = F.max_pool2d(x, (2,2))
       #128, 2, 2
       x = x.view(-1, 128 * 2 * 2)
       x = F.relu(self.fc1(x))
       x = self.fc2(x)
       return F.log_softmax(x, dim=-1)


In [7]:
# Summary to check torch model structure
from torchsummary import summary
resnet = FCNet()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
resnet = resnet.to(device)
summary(resnet, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 28, 24, 24]             728
            Conv2d-2           [-1, 42, 12, 12]          10,626
            Linear-3                  [-1, 512]         774,656
            Linear-4                   [-1, 10]           5,130
Total params: 791,140
Trainable params: 791,140
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.17
Params size (MB): 3.02
Estimated Total Size (MB): 3.19
----------------------------------------------------------------


In [75]:
FLAGS = flags.FLAGS
NB_EPOCHS = 2
BATCH_SIZE = 128
LEARNING_RATE = .001

def trainTorch(torch_model, train_loader, test_loader,
        nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE, train_end=-1, test_end=-1, learning_rate=LEARNING_RATE):


    # Truncate the datasets so that our test run more quickly
  #   train_loader.dataset.train_data = train_loader.dataset.train_data[:train_end]
  #   test_loader.dataset.test_data = test_loader.dataset.test_data[:test_end]

    # Train our model
    optimizer = optim.Adam(torch_model.parameters(), lr=learning_rate)
    train_loss = []

    total = 0
    correct = 0
    step = 0
    # breakstep = 0
    for _epoch in range(nb_epochs):
      # if breakstep == 2:
      #     # print("break all!")
      #     break
      for xs, ys in train_loader:
        xs, ys = Variable(xs), Variable(ys)
        if torch.cuda.is_available():
          xs, ys = xs.cuda(), ys.cuda()
        optimizer.zero_grad()
        preds = torch_model(xs)
        loss = F.nll_loss(preds, ys)
        loss.backward()  # calc gradients
        train_loss.append(loss.data.item())
        optimizer.step()  # update gradients

        preds_np = preds.cpu().detach().numpy()
        correct += (np.argmax(preds_np, axis=1) == ys.cpu().detach().numpy()).sum()
        total += train_loader.batch_size
        step += 1
        if total % 1000 == 0:
          acc = float(correct) / total
          print('[%s] Training accuracy: %.2f%%' % (step, acc * 100))
          total = 0
          correct = 0
          # breakstep += 1
          # if breakstep == 2:
          #     # print("break!")
          #     break
    
def Ensembler(preds):
    finalPred = np.zeros(len(preds[0]))
    for i in range(len(preds[0])):
      scoreList = np.zeros(10)
      for pred in preds:
        scoreList[pred[i]] += 1
      finalPred[i] = np.argmax(scoreList)
    return finalPred

def evalClean(model1=None, model2=None, model3=None, model4=None, test_loader=None, report=None, singleModel=0):
    if singleModel:
      print("Evaluating single model results on clean data")
    else:
      print("Evaluating the ensembled method on clean data")
    total = 0
    correct = 0
    with torch.no_grad():
      model1.eval()
      if not singleModel:
        model2.eval()
        model3.eval()
        # only when 4 models
        # model4.eval()
      for xs, ys in test_loader:
        xs, ys = Variable(xs), Variable(ys)
        if torch.cuda.is_available():
          xs, ys = xs.cuda(), ys.cuda()
        preds1 = model1(xs)
        preds_np1 = preds1.cpu().detach().numpy()
        if not singleModel:
          preds2 = model2(xs)
          preds_np2 = preds2.cpu().detach().numpy()
          preds3 = model3(xs)
          preds_np3 = preds3.cpu().detach().numpy()
          # only when 4 models
          # preds4 = model4(xs)
          # preds_np4 = preds4.cpu().detach().numpy()    
          #preds for 3 and 4
          preds = [np.argmax(preds_np1, axis=1), np.argmax(preds_np2, axis=1), np.argmax(preds_np3, axis=1)]
          # preds = [np.argmax(preds_np1, axis=1), np.argmax(preds_np2, axis=1), np.argmax(preds_np3, axis=1), np.argmax(preds_np4, axis=1)]
          finalPred = Ensembler(preds)
        else:
          finalPred = np.argmax(preds_np1, axis=1)
        correct += (finalPred == ys.cpu().detach().numpy()).sum()
        total += len(xs)
    acc = float(correct) / total
    report.clean_train_clean_eval = acc
    print('Clean accuracy: %.2f%%' % (acc * 100))

    return report

def evalAdvAttack(fgsm_model=None, model2=None, model3=None, model4=None, test_loader=None, singleModel=0):
    # fgsm_model
    # fast_gradient_method(model_fn, x, eps, norm, clip_min=None, clip_max=None, y=None, targeted=False, sanity_checks=False):
    if singleModel:
      print("Evaluating single model results on adv data")
    else:
      print("Evaluating the ensembled method on adv data")
    total = 0
    correct = 0
    fgsm_model.eval()
    if not singleModel:
      model2.eval()
      model3.eval()
      # only when 4 models
      # model4.eval()
    
    for xs, ys in test_loader:
      if torch.cuda.is_available():
        xs, ys = xs.cuda(), ys.cuda()
      #pytorch fast gradient method
      xs = fast_gradient_method(fgsm_model, xs, eps=0.3, norm=np.inf, clip_min=0., clip_max=1.)
      # xs = fast_gradient_method(fgsm_model, xs, eps=0.3, norm=np.inf)
      xs, ys = Variable(xs), Variable(ys)
      preds1 = fgsm_model(xs)
      # loss = F.nll_loss(preds1, ys)
      # loss.backward()  # calc gradients
      # train_loss.append(loss.data.item())
      # optimizer.step()  # update gradients
      preds_np1 = preds1.cpu().detach().numpy()
      if not singleModel:
          preds2 = model2(xs)
          preds_np2 = preds2.cpu().detach().numpy()
          preds3 = model3(xs)
          preds_np3 = preds3.cpu().detach().numpy()
          # only when 4 models
          # preds4 = model4(xs)
          # preds_np4 = preds4.cpu().detach().numpy()  
          #preds for 3 and 4
          preds = [np.argmax(preds_np1, axis=1), np.argmax(preds_np2, axis=1), np.argmax(preds_np3, axis=1)]
          # preds = [np.argmax(preds_np1, axis=1), np.argmax(preds_np2, axis=1), np.argmax(preds_np3, axis=1), np.argmax(preds_np4, axis=1)]
          finalPred = Ensembler(preds)
      else:
          finalPred = np.argmax(preds_np1, axis=1)
      correct += (finalPred == ys.cpu().detach().numpy()).sum()
      total += test_loader.batch_size
    acc = float(correct) / total
    print('Adv accuracy: {:.3f}％'.format(acc * 100))
      # break

def evalCombined(model1, model2, model3, test_loader, report):
    # This function evaluates the main model (model 1) and the ensembled data (model1/2/3) on clean and adv data

    # Evaluate single model on clean data
    evalClean(model1=model1, test_loader=test_loader, report=report, singleModel=1)
    # Evaluate ensembled model on clean data
    evalClean(model1=model1, model2=model2, model3=model3, test_loader=test_loader, report=report, singleModel=0)
    # Evaluate single model on adversarial attack
    evalAdvAttack(fgsm_model=model1, test_loader=test_loader, singleModel=1)
    # Evaluate ensembled model on adversarial attack
    evalAdvAttack(fgsm_model=model1, model2=model2, model3=model3, test_loader=test_loader, singleModel=0)

def advTrain(torch_model, train_loader, test_loader,
        nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE, train_end=-1, test_end=-1, learning_rate=LEARNING_RATE):
    optimizer = optim.Adam(torch_model.parameters(), lr=learning_rate)
    train_loss = []
    total = 0
    correct = 0
    totalAdv = 0
    correctAdv = 0
    step = 0
    # breakstep = 0
    for _epoch in range(nb_epochs):
      for xs, ys in train_loader:
        #Normal Training
        xs, ys = Variable(xs), Variable(ys)
        if torch.cuda.is_available():
          xs, ys = xs.cuda(), ys.cuda()
        optimizer.zero_grad()
        preds = torch_model(xs)
        loss = F.nll_loss(preds, ys)
        loss.backward()  # calc gradients
        train_loss.append(loss.data.item())
        optimizer.step()  # update gradients
        preds_np = preds.cpu().detach().numpy()
        correct += (np.argmax(preds_np, axis=1) == ys.cpu().detach().numpy()).sum()
        total += train_loader.batch_size

        #Adversarial Training
        xs = fast_gradient_method(torch_model, xs, eps=0.3, norm=np.inf, clip_min=0., clip_max=1.)
        xs, ys = Variable(xs), Variable(ys)
        if torch.cuda.is_available():
          xs, ys = xs.cuda(), ys.cuda()
        optimizer.zero_grad()
        preds = torch_model(xs)
        loss = F.nll_loss(preds, ys)
        loss.backward()  # calc gradients
        train_loss.append(loss.data.item())
        optimizer.step()  # update gradients
        preds_np = preds.cpu().detach().numpy()
        correctAdv += (np.argmax(preds_np, axis=1) == ys.cpu().detach().numpy()).sum()
        totalAdv += train_loader.batch_size
        
        step += 1
        if total % 1000 == 0:
          acc = float(correct) / total
          print('[%s] Clean Training accuracy: %.2f%%' % (step, acc * 100))
          total = 0
          correct = 0
          accAdv = float(correctAdv) / totalAdv
          print('[%s] Adv Training accuracy: %.2f%%' % (step, accAdv * 100))
          totalAdv = 0
          correctAdv = 0




In [84]:
# Experiment with 3 models
# model1 = LeNet5()
# model2 = FCNet()
# model3 = PytorchMnistModel()

# # Experiment 1
# model1 = PytorchMnistModel()
# model2 = LeNet5()
# model3 = ResNet1()
# model4 = VGGNet()

#Experiment 2
model1 = ResNet1()
model2 = ResNet1()
model3 = ResNet1()
# model4 = ResNet1()

# #Experiment 3
# model1 = ResNet1()
# model2 = ResNet2()
# model3 = ResNet3()
# model4 = ResNet4()

if torch.cuda.is_available():
  model1 = model1.cuda()
  model2 = model2.cuda()
  model3 = model3.cuda()
  #model4 = model4.cuda()


In [85]:
#Initialize train loader and test loader and hyperparameters
nb_epochs = 4
batch_size = 128
learning_rate = 0.001
train_end = -1
test_end = -1
report = AccuracyReport()
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                    transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size)

In [86]:
#Training
print("Training Model 1")
trainTorch(model1, train_loader, test_loader, nb_epochs, batch_size, train_end, test_end, learning_rate)
print("Training Model 2")
trainTorch(model2, train_loader, test_loader, nb_epochs, batch_size, train_end, test_end, learning_rate)
print("Training Model 3")
trainTorch(model3, train_loader, test_loader, nb_epochs, batch_size, train_end, test_end, learning_rate)

Training Model 1
[125] Training accuracy: 85.12%
[250] Training accuracy: 96.45%
[375] Training accuracy: 97.59%
[500] Training accuracy: 97.74%
[625] Training accuracy: 98.45%
[750] Training accuracy: 98.24%
[875] Training accuracy: 98.37%
[1000] Training accuracy: 98.38%
[1125] Training accuracy: 99.05%
[1250] Training accuracy: 98.91%
[1375] Training accuracy: 98.78%
[1500] Training accuracy: 98.88%
[1625] Training accuracy: 99.23%
[1750] Training accuracy: 98.94%
[1875] Training accuracy: 98.90%
Training Model 2
[125] Training accuracy: 85.81%
[250] Training accuracy: 96.55%
[375] Training accuracy: 97.34%
[500] Training accuracy: 97.67%
[625] Training accuracy: 98.38%
[750] Training accuracy: 98.28%
[875] Training accuracy: 98.29%
[1000] Training accuracy: 98.64%
[1125] Training accuracy: 98.72%
[1250] Training accuracy: 98.72%
[1375] Training accuracy: 98.78%
[1500] Training accuracy: 98.74%
[1625] Training accuracy: 99.08%
[1750] Training accuracy: 99.01%
[1875] Training accurac

In [81]:
print("Evaluation before training")
print("Evaluating model 1")
evalCombined(model1=model1, model2=model2, model3=model3, test_loader=test_loader,report=report)
print("Evaluating model 2")
evalCombined(model1=model2, model2=model1, model3=model3, test_loader=test_loader,report=report)
print("Evaluating model 3")
evalCombined(model1=model3, model2=model1, model3=model2, test_loader=test_loader,report=report)

Evaluation before training
Evaluating model 1
Evaluating single model results on clean data
Clean accuracy: 98.80%
Evaluating the ensembled method on clean data
Clean accuracy: 99.27%
Evaluating single model results on adv data
Adv accuracy: 1.483％
Evaluating the ensembled method on adv data
Adv accuracy: 13.360％
Evaluating model 2
Evaluating single model results on clean data
Clean accuracy: 99.01%
Evaluating the ensembled method on clean data
Clean accuracy: 99.27%
Evaluating single model results on adv data
Adv accuracy: 3.016％
Evaluating the ensembled method on adv data
Adv accuracy: 7.941％
Evaluating model 3
Evaluating single model results on clean data
Clean accuracy: 99.05%
Evaluating the ensembled method on clean data
Clean accuracy: 99.27%
Evaluating single model results on adv data
Adv accuracy: 2.373％
Evaluating the ensembled method on adv data
Adv accuracy: 11.116％


In [82]:
print("Perform Adversarial Training")
advTrain(model1, train_loader, test_loader, nb_epochs, batch_size, train_end, test_end, learning_rate)
advTrain(model2, train_loader, test_loader, nb_epochs, batch_size, train_end, test_end, learning_rate)
advTrain(model3, train_loader, test_loader, nb_epochs, batch_size, train_end, test_end, learning_rate)

Perform Adversarial Training
[125] Clean Training accuracy: 97.07%
[125] Adv Training accuracy: 35.04%
[250] Clean Training accuracy: 97.56%
[250] Adv Training accuracy: 61.61%
[375] Clean Training accuracy: 97.98%
[375] Adv Training accuracy: 72.96%
[500] Clean Training accuracy: 97.83%
[500] Adv Training accuracy: 80.51%
[625] Clean Training accuracy: 98.26%
[625] Adv Training accuracy: 84.45%
[750] Clean Training accuracy: 98.20%
[750] Adv Training accuracy: 86.99%
[875] Clean Training accuracy: 98.11%
[875] Adv Training accuracy: 87.79%
[1000] Clean Training accuracy: 98.36%
[1000] Adv Training accuracy: 89.32%
[1125] Clean Training accuracy: 98.47%
[1125] Adv Training accuracy: 89.33%
[1250] Clean Training accuracy: 98.63%
[1250] Adv Training accuracy: 91.11%
[1375] Clean Training accuracy: 98.36%
[1375] Adv Training accuracy: 91.41%
[1500] Clean Training accuracy: 98.66%
[1500] Adv Training accuracy: 92.33%
[1625] Clean Training accuracy: 98.71%
[1625] Adv Training accuracy: 92.5

In [83]:
print("Evaluation before training")
print("Evaluating model 1")
evalCombined(model1=model1, model2=model2, model3=model3, test_loader=test_loader,report=report)
print("Evaluating model 2")
evalCombined(model1=model2, model2=model1, model3=model3, test_loader=test_loader,report=report)
print("Evaluating model 3")
evalCombined(model1=model3, model2=model1, model3=model2, test_loader=test_loader,report=report)

Evaluation before training
Evaluating model 1
Evaluating single model results on clean data
Clean accuracy: 98.54%
Evaluating the ensembled method on clean data
Clean accuracy: 99.25%
Evaluating single model results on adv data
Adv accuracy: 90.180％
Evaluating the ensembled method on adv data
Adv accuracy: 87.955％
Evaluating model 2
Evaluating single model results on clean data
Clean accuracy: 98.95%
Evaluating the ensembled method on clean data
Clean accuracy: 99.25%
Evaluating single model results on adv data
Adv accuracy: 94.531％
Evaluating the ensembled method on adv data
Adv accuracy: 88.430％
Evaluating model 3
Evaluating single model results on clean data
Clean accuracy: 99.10%
Evaluating the ensembled method on clean data
Clean accuracy: 99.25%
Evaluating single model results on adv data
Adv accuracy: 96.994％
Evaluating the ensembled method on adv data
Adv accuracy: 87.876％


In [61]:
# def AttackOnModel(fgsm_model=None, test_model1=None, test_model2=None, test_model3=None, test_loader=None, report=None, singleModel=0):
#     # We use tf for evaluation on adversarial data
#     sess = tf.Session()

#     x_op = tf.placeholder(tf.float32, shape=(None, 1, 28, 28,))

#     # Convert pytorch model to a tf_model and wrap it in cleverhans
#     tf_model_fn = convert_pytorch_model_to_tf(fgsm_model)
#     cleverhans_model = CallableModelWrapper(tf_model_fn, output_layer='logits')

#     # Convert the testing models
#     test_model_fn1 = convert_pytorch_model_to_tf(test_model1)
#     test_model_fn2 = convert_pytorch_model_to_tf(test_model2)
#     test_model_fn3 = convert_pytorch_model_to_tf(test_model3)

#     # Create an FGSM attack
#     fgsm_op = FastGradientMethod(cleverhans_model, sess=sess)
#     fgsm_params = {'eps': 0.3,
#                    'clip_min': 0.,
#                    'clip_max': 1.}
#     adv_x_op = fgsm_op.generate(x_op, **fgsm_params)

#     adv_preds_op0 = tf_model_fn(adv_x_op)
#     adv_preds_op1 = test_model_fn1(adv_x_op)
#     adv_preds_op2 = test_model_fn2(adv_x_op)
#     adv_preds_op3 = test_model_fn3(adv_x_op)
#     # Run an evaluation of our model against fgsm
#     total = 0
#     correct = 0
#     for xs, ys in test_loader:
#       adv_preds0 = sess.run(adv_preds_op0, feed_dict={x_op: xs})
#       adv_preds1 = sess.run(adv_preds_op1, feed_dict={x_op: xs})
#       adv_preds2 = sess.run(adv_preds_op2, feed_dict={x_op: xs})
#       adv_preds3 = sess.run(adv_preds_op3, feed_dict={x_op: xs})
#       preds = [np.argmax(adv_preds0, axis=1), np.argmax(adv_preds1, axis=1), np.argmax(adv_preds2, axis=1), np.argmax(adv_preds3, axis=1)]
#       if not singleModel:
#         finalPred = Ensembler(preds)
#       else:
#         finalPred = np.argmax(adv_preds1, axis=1)
#       correct += (finalPred == ys.cpu().detach().numpy()).sum()
#       total += test_loader.batch_size

#     acc = float(correct) / total
#     print('Adv accuracy: {:.3f}％'.format(acc * 100))
#     report.clean_train_adv_eval = acc
#     # train(sess, loss, x_train, y_train, evaluate=evaluate, args=train_params, rng=rng, var_list=model.get_params())
#     return report